In [15]:
import pandas as pd
from pandas import DataFrame
from pandas import concat
import matplotlib.pyplot as plt
import xgboost as xgb
import numpy
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from numpy import argmax
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import random
%matplotlib inline

inPath = './datas/first_train_data_20180131/'


In [16]:
train_index = pd.read_csv('./datas/first_train_index_20180131.csv')
label = train_index.type
encoder = LabelEncoder()
encoder.fit(label)

LabelEncoder()

In [17]:
test_index = pd.read_csv('./datas/first_test_index_20180131.csv')

In [30]:
def genearte_batch_data(maxFileIndex, batchSize=32, sample_size=1000):
    global encoder
    i = 0
    while True:
        i = i % (maxFileIndex+1)
        fileName = './train_feature_all_'+ str(i) +'.csv'
        print(fileName)
        train_set = pd.read_csv(fileName, header=None)
        renameCol = {0:'id'}
        renameCol[train_set.shape[1]-1] = 'label'
        train_set.rename(columns=renameCol, inplace=True)
        train_star = train_set[train_set.label=='star'].sample(sample_size, replace=True)
        unknow_star = train_set[train_set.label=='unknown'].sample(sample_size, replace=True)
        galaxy_star = train_set[train_set.label=='galaxy'].sample(sample_size, replace=True)
        qso_star = train_set[train_set.label=='qso'].sample(sample_size, replace=True)
        sampled_train = pd.concat([train_star,unknow_star,galaxy_star, qso_star])
        sampled_train = sampled_train.sample(sampled_train.shape[0], replace=False)
#         y = sampled_train.label
#         encoded_Y = encoder.transform(y)
#         dummy_y = np_utils.to_categorical(encoded_Y)
#         yield sampled_train.iloc[:,1:-1], dummy_y
        for j in range(0, sample_size*4/batchSize):
            batch_train = sampled_train.iloc[(j*batchSize):(j+1)*batchSize, :]
            x = batch_train.iloc[:,1:-1]
            y = batch_train.label
            encoded_Y = encoder.transform(y)
            dummy_y = np_utils.to_categorical(encoded_Y)
            yield x, dummy_y
        i = i + 1

In [31]:
def genearte_test_batch_data(maxFileIndex):
    i = 0
    while True:
        if i > maxFileIndex:
            break
        i = i % (maxFileIndex+1)
        print(i)
        test_set = pd.read_csv('./test_feature_all_'+ str(i) +'.csv', header=None)
        renameCol = {0:'id'}
        test_set.rename(columns=renameCol, inplace=True)
        x = test_set.iloc[:,1:-1]
        i = i + 1
        yield x

In [42]:
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(40, input_dim=2600, activation='relu'))
	model.add(Dense(4, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
	return model

In [ ]:
maxFileIndex=48
epochs = 50
batchSize= 2000
sample_size = 1000
steps_per_epoch = sample_size*4*(maxFileIndex+1)/batchSize*10
model = baseline_model()
bst = model.fit_generator(genearte_batch_data(maxFileIndex,batchSize), steps_per_epoch=steps_per_epoch, epochs=epochs, workers=10)


./train_feature_all_0.csv
Epoch 1/50
./train_feature_all_1.csv
980/980 [==============================] - 4384s 4s/step - loss: 10.2971 - acc: 0.3570
Epoch 2/50
./train_feature_all_1.csv
980/980 [==============================] - 4437s 5s/step - loss: 9.8646 - acc: 0.3856
Epoch 3/50
./train_feature_all_1.csv
980/980 [==============================] - 4309s 4s/step - loss: 9.6269 - acc: 0.4007
Epoch 4/50
./train_feature_all_1.csv
980/980 [==============================] - 4312s 4s/step - loss: 9.2721 - acc: 0.4230
Epoch 5/50
./train_feature_all_1.csv
980/980 [==============================] - 4309s 4s/step - loss: 9.7075 - acc: 0.3966
Epoch 6/50
./train_feature_all_1.csv
980/980 [==============================] - 4307s 4s/step - loss: 8.7680 - acc: 0.4544
Epoch 7/50
./train_feature_all_1.csv
980/980 [==============================] - 4311s 4s/step - loss: 9.8471 - acc: 0.3882
Epoch 8/50
./train_feature_all_1.csv
980/980 [==============================] - 4311s 4s/step - loss: 9.4418 - a

In [303]:
maxFileIndex = 9
steps = maxFileIndex+1;
pred = model.predict_generator(genearte_test_batch_data(maxFileIndex), steps=steps, workers=4)


In [301]:
batch_size

20

In [304]:
def submit(preds, test_feature, fileName):
    submission = test_feature[['id']]
    final_pred = encoder.inverse_transform(argmax(preds,axis=1))
    submission['pred'] = pd.Series(final_pred)
    submission.to_csv('./predicts/'+fileName, header=False, index=False)
    return submission

In [306]:
rs  =submit(pred, test_index, '0219_2_dnn.csv')

array(['star', 'galaxy', 'unknown', 'qso'], dtype=object)

array(['star', 'star', 'star', 'star', 'star', 'star', 'star', 'star',
       'star', 'star', 'star', 'star', 'star', 'star', 'star', 'star',
       'star', 'star', 'star', 'star', 'star', 'star', 'star', 'star',
       'star', 'star', 'star', 'star', 'star', 'star', 'star', 'star',
       'star', 'star', 'star', 'star', 'star', 'star', 'star', 'star',
       'star', 'star', 'star', 'star', 'star', 'star', 'star', 'star',
       'star', 'star', 'star', 'star', 'star', 'star', 'star', 'star',
       'star', 'star', 'star', 'star', 'star', 'star', 'star', 'star',
       'star', 'star', 'star', 'star', 'star', 'star', 'star', 'star',
       'star', 'star', 'star', 'star', 'star', 'star', 'star', 'star',
       'star', 'star', 'star', 'star', 'star', 'star', 'star', 'star',
       'star', 'star', 'star', 'star', 'star', 'star', 'star', 'star',
       'star', 'star', 'star', 'star'], dtype=object)

In [158]:
genearte_batch_data(train_index,5)

<generator object genearte_batch_data at 0x1a55d43e10>

In [106]:
X = train_d.iloc[:,:-1]
seed = 7
numpy.random.seed(seed)
estimator = KerasClassifier(build_fn=baseline_model, epochs=1000,  verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))





Baseline: 48.10% (42.55%)


array([ 0.02,  0.03,  0.08,  0.9 ,  0.9 ,  0.93,  0.91,  0.08,  0.89,  0.07])